# 07. TP Mode Selection and Cross Validation

#### enseignant: Anastasios Giovanidis 2020 - 2021
#### date: 18 Novembre 2020

#### student name or binome:

This is the TP related to mode selection and cross-validation. We will focus here on regression only, but the techniques can be generalised for classification.
We will need to import the following libraries.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import random

In [10]:
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_validate, Bootstrap
from sklearn.metrics import mean_squared_error, r2_score

## Exercise 1  (Polynomial Regression)

We will now work on problems of polynomial regression. As a means of example, we will try to fit data sampled from the following curve:

$y = 4 + 2x + 0.5x^2 - 0.07x^3 + \epsilon$.

First create synthetic data for $(x_1,x_2,x_3,y,Er)$. The regression plane (>2 dimensions) is given by the following expression, for $\ell$ unknown features:

$\hat{y} = \hat{\beta}_0 + \hat{\beta}_1x + \hat{\beta}_2x^2 +\ldots + \hat{\beta}_{\ell}x^{\ell}$.

**We will use again a multi-linear regression model, to estimate the unknowns.** 
We propose to use regression models with different polynomial degree, because in reality we do not know which is the appropriate model. 

**Question:**
* We start with a linear model of degree $\ell=3$. Split the total dataset of $n=100$ in a train set and a test set (80%-20%). Plot the least-squares curve and output the $R^2$ value. (use $sigmae = 5$, $xl=0, xh = 10$, $n=200$)

* Repeat the process for $\ell = 1, 2, 3, 6, 9, 16$. 

(a) plot the $MSE$ value versus $\ell$ for the Train data. 
(b) plot the $MSE$ value versu2 $\ell$ for the Test data. What do you observe?

(c) plot the $R^2$ value versus $\ell$ for the Train data. 
(d) plot the $R^2$ value versus $\ell$ for the Test data. What do you observe?

Use sklearn commands: 

train_test_split, 
linear_model.LinearRegression(), 
mean_squared_error, 
r2_score

**Build synthetic data**

In [3]:
n = 200
b0 = 4
b1 = np.array([2,0.5,-0.07])
mue, sigmae = 0, 5
xl, xh = 0, 10
np.random.seed(199)
Er = np.random.normal(mue, sigmae, n)
np.random.seed(199)
x0 = np.random.uniform(xl,xh,n)
x = np.array([x0])
x = np.append(x,np.array([x0**2]),axis=0)
x = np.append(x,np.array([x0**3]),axis=0)
#
y = b0 + b1[0]*x[0]+ b1[1]*x[1]+ b1[2]*x[2]+Er

Here is the datasheet that is given to you: 
**From this point on you don't know how this was produced, you just have the data on x and y and are asked to do a regression.**

In [5]:
dataSynth = {'x': x[0],'y': y}
df = pd.DataFrame(data=dataSynth)
df

,x,y
0,6.547917,24.421765
1,9.820393,3.953867
2,3.999047,26.961203
3,8.441526,5.177844
4,4.638424,9.378242
...,...,...
195,3.032599,8.649734
196,6.540144,19.545533
197,6.284375,20.354621
198,5.722965,23.608383


**Solution:**

## Exercise 2 - Cross Validation

Use the following toy dataset:

$D_6 = \left\{(1,3),\ (2,4),\ (3,8),\ (4,9), \ (5,12),\ (7,14) \right\}$.

**Question A:** Find all produced train-test splits when applying

- (a) LOOCV.  Command: LeaveOneOut()

- (b) 3-fold CV method. Command: KFold(n_splits)

- (c) Bootstrap method (with 5 bootstraps). Command Bootstrap

**Question B:** Perform linear regression and find the average $MSE_{test}$ for the data sets

- (d) using 2-fold CV method. Command: cross_validate

**Part II:** Apply the same method to the data set below for the **TV-sales** pair:

The dataset "Advertising.csv" can be downloaded from: http://faculty.marshall.usc.edu/gareth-james/ISL/data.html

In [16]:
directory = #your directory
prefix = "Advertising.csv"
filename1 = directory+prefix
dataAd = np.loadtxt(filename1, delimiter=",",skiprows=1,usecols=[1,2,3,4])

In [7]:
# pandas
pdAd = pd.DataFrame(dataAd, columns=["TV","radio","newspaper","sales"])

In [8]:
TV = pdAd.iloc[:,0].values
#radio = pdAd.iloc[:,1].values
#news = pdAd.iloc[:,2].values
sales = pdAd.iloc[:,3].values

**Answer:**